In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Pré-Processamento

## Crop das Imagens

In [ ]:
# diretório que contém as imagens originais
input_dir = '/content/drive/MyDrive/modulo10/data/dataset_inteli_crop/masks'

# diretório onde as imagens cortadas serão salvas
output_dir = '/content/drive/MyDrive/modulo10/data/dataset_inteli_crop/testeImagensCortadasMasks'

os.makedirs(output_dir, exist_ok=True)

# dimensões dos cortes
crop_width, crop_height = 120, 120

# método para cortar uma imagem em pedaços menores
def crop_image(image_path, output_dir, crop_width, crop_height):
    img = Image.open(image_path)
    img_width, img_height = img.size

    for i in range(0, img_width, crop_width):
        for j in range(0, img_height, crop_height):
            box = (i, j, i + crop_width, j + crop_height)
            cropped_img = img.crop(box)
            cropped_img_path = os.path.join(output_dir, f"{os.path.basename(image_path).split('.')[0]}_{i}_{j}.png")
            cropped_img.save(cropped_img_path)

for filename in os.listdir(input_dir):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(input_dir, filename)
        crop_image(image_path, output_dir, crop_width, crop_height)

In [ ]:
from PIL import Image
import os
import uuid

# diretórios contendo as imagens originais
input_image_dirs = [
    '/content/drive/MyDrive/modulo10/data/dataset_inteli_crop/marked_rgbs',
    '/content/drive/MyDrive/modulo10/data/dataset_inteli_crop/rgbs',
    '/content/drive/MyDrive/modulo10/data/dataset_inteli_crop/tci_tifs',
    '/content/drive/MyDrive/modulo10/data/dataset_inteli_crop/tci_pngs',
]
# diretório contendo as máscaras
input_mask_dir = '/content/drive/MyDrive/modulo10/data/dataset_inteli_crop/masks'

# diretórios onde as imagens e máscaras cortadas serão salvas
output_image_dir = '/content/drive/MyDrive/modulo10/data/dataset_inteli_crop/newTargetImages2'
output_mask_dir = '/content/drive/MyDrive/modulo10/data/dataset_inteli_crop/newTargetMasks2'

os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_mask_dir, exist_ok=True)

# dimensões dos cortes
crop_width, crop_height = 120, 120

# cortes por imagem
max_cuts = 5

# função para cortar a imagem e sua máscara em pedaços menores
def crop_image_and_mask(image_path, mask_path, output_image_dir, output_mask_dir, crop_width, crop_height, max_cuts):
    img = Image.open(image_path)
    mask = Image.open(mask_path)
    img_width, img_height = img.size

    # gera um nome aleatório para a imagem
    base_name = str(uuid.uuid4())[:8]

    ext = os.path.splitext(image_path)[1]

    cut_count = 0
    for i in range(0, img_width, crop_width):
        for j in range(0, img_height, crop_height):
            if cut_count >= max_cuts:
                return
            box = (i, j, i + crop_width, j + crop_height)
            cropped_img = img.crop(box)
            cropped_mask = mask.crop(box)
            cropped_img_path = os.path.join(output_image_dir, f"{base_name}_{cut_count+1}{ext}")
            cropped_mask_path = os.path.join(output_mask_dir, f"{base_name}_{cut_count+1}{ext}")
            cropped_img.save(cropped_img_path)
            cropped_mask.save(cropped_mask_path)
            cut_count += 1

# percorrer todos os diretórios de entrada
for input_image_dir in input_image_dirs:
    for filename in os.listdir(input_image_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff')):
            image_path = os.path.join(input_image_dir, filename)
            mask_path = os.path.join(input_mask_dir, filename)  # assumindo que as máscaras têm o mesmo nome que as imagens
            if os.path.exists(mask_path):  # garantindo que a máscara correspondente existe
                crop_image_and_mask(image_path, mask_path, output_image_dir, output_mask_dir, crop_width, crop_height, max_cuts)

# Modelo de Segmentação de Imagens com U-Net e VGG16

## Descrição Geral
Este notebook implementa um modelo de segmentação de imagens utilizando a arquitetura U-Net com backbone VGG16. A segmentação é realizada em imagens e máscaras carregadas de um diretório específico. A abordagem é útil para tarefas como segmentação de áreas agrícolas em imagens de satélite.


## Habilitando GPU no Google Colab
Para habilitar a GPU no Google Colab, siga os passos abaixo:

1. Vá para `Tempo de execução` ou `Runtime` no menu.
2. Selecione `Alterar tipo de tempo de execução` ou `Change runtime`.
3. Na janela que abrir, escolha `GPU` no menu suspenso `Acelerador de hardware`, recomendamos o uso da A100 pela capacidade mista de CPU e GPU.

Após realizar esses passos, a GPU será habilitada para o notebook.

## Importando libs necessárias

In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

## Verificando se a GPU está sendo usada
Com o código abaixo é possível verificar se a GPU está sendo usada pelo TensorFlow.

In [ ]:
# Verifica se a GPU está disponível
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs disponíveis: {gpus}")
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("Nenhuma GPU disponível.")

# Verifica se o TensorFlow está utilizando a GPU
if tf.test.gpu_device_name():
    print('GPU ativa:', tf.test.gpu_device_name())
else:
    print("Nenhuma GPU ativa.")

## Preparação de Dados

A preparação de dados envolve carregar as imagens e as máscaras correspondentes, normalizando e transformando esses dados para o formato adequado para treinamento. As imagens são normalizadas para o intervalo [0, 1], e as máscaras são binarizadas, onde valores acima de 0.5 são considerados 1 (objeto de interesse) e abaixo são 0 (fundo).


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

def load_images_and_masks_in_batches(data_dir, mask_dir, batch_size=51):
    file_names = sorted(os.listdir(data_dir))
    images = []
    masks = []
    count = 0

    for file in file_names:
        if file.endswith('.png'):
            img_path = os.path.join(data_dir, file)
            mask_path = os.path.join(mask_dir, file)
            try:
                img = load_img(img_path)  # Carrega a imagem no formato padrão
                img = img_to_array(img) / 255.0  # Normaliza a imagem para o intervalo [0, 1]
                mask = load_img(mask_path, color_mode='grayscale')  # Carrega a máscara como grayscale
                mask = img_to_array(mask) / 255.0  # Normaliza a máscara
                mask = (mask > 0.5).astype(np.float32)  # Binariza a máscara

                if img.shape == (120, 120, 3) and mask.shape == (120, 120, 1):
                    images.append(img)
                    masks.append(mask[:, :, 0])  # Garante que a máscara seja um array 2D
                    count += 1
                else:
                    print(f"Descartado por dimensões incorretas: {file}")

                if count == batch_size:
                    yield np.array(images), np.array(masks)
                    images, masks = [], []
                    count = 0
            except Exception as e:
                print(f"Erro ao carregar {file}: {e}")

    if images and masks:
        yield np.array(images), np.array(masks)

# Caminhos para os diretórios de imagens e máscaras
data_dir = '/content/drive/MyDrive/modulo10/data/dataset_inteli_crop/newTargetImages2'
mask_dir = '/content/drive/MyDrive/modulo10/data/dataset_inteli_crop/newTargetMasks2'

# Processar imagens e máscaras em batches
all_images = []
all_masks = []
batch_size = 51
for batch_images, batch_masks in load_images_and_masks_in_batches(data_dir, mask_dir, batch_size):
    all_images.extend(batch_images)
    all_masks.extend(batch_masks)

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(all_images, all_masks, test_size=0.1, random_state=42)


## Função de perda personalizada

Definimos uma função de perda personalizada que combina a perda Dice e a perda binária de entropia cruzada para otimizar o modelo.

In [ ]:
def dice_loss(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)

def combined_loss(y_true, y_pred):
    return tf.keras.losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

## Arquitetura do modelo

### Descrição Do Modelo Utilizado - Estrutura Geral

O modelo U-Net é uma rede neural convolucional projetada para tarefas de segmentação de imagem. A arquitetura é composta por duas partes principais: o codificador (encoder) e o decodificador (decoder). A estrutura do U-Net permite capturar tanto o contexto global quanto os detalhes locais da imagem. O U-Net, proposto por Ronneberger et al. (2015), destaca-se pelo uso de conexões de skip entre o encoder e o decoder, que preservam informações de alta resolução essenciais para segmentação precisa. A extensão 3D U-Net, proposta por Çiçek et al. (2016), melhora ainda mais a capacidade do modelo em aplicações volumétricas. Ele é amplamente utilizado em diversas aplicações, incluindo a segmentação de tumores e análise de imagens médicas.

Referências:

Ronneberger, O., Fischer, P., & Brox, T. (2015). U-Net: Convolutional Networks for Biomedical Image Segmentation. In Medical Image Computing and Computer-Assisted Intervention – MICCAI 2015 (pp. 234-241).

Çiçek, Ö., et al. (2016). 3D U-Net: Learning Dense Volumetric Segmentation from Sparse Annotation. In Medical Image Computing and Computer-Assisted Intervention – MICCAI 2016 (pp. 424-432).


### Codificador (Encoder)


O codificador é baseado na arquitetura VGG16 pré-treinada no conjunto de dados ImageNet. Ele consiste em uma série de camadas convolucionais seguidas por camadas de pooling. A função do codificador é extrair características da imagem de entrada em diferentes níveis de abstração. As camadas convolucionais e de pooling são:


- **Block 1:**
  - 2 camadas Conv2D com 64 filtros e kernel 3x3, ativação ReLU
  - 1 camada MaxPooling2D com tamanho de pool 2x2

- **Block 2:**
  - 2 camadas Conv2D com 128 filtros e kernel 3x3, ativação ReLU
  - 1 camada MaxPooling2D com tamanho de pool 2x2

- **Block 3:**
  - 3 camadas Conv2D com 256 filtros e kernel 3x3, ativação ReLU
  - 1 camada MaxPooling2D com tamanho de pool 2x2

- **Block 4:**
  - 3 camadas Conv2D com 512 filtros e kernel 3x3, ativação ReLU
  - 1 camada MaxPooling2D com tamanho de pool 2x2

- **Block 5:**
  - 3 camadas Conv2D com 512 filtros e kernel 3x3, ativação ReLU

### Decodificador (Decoder)
O decodificador reconstrói a imagem de saída a partir das características extraídas pelo codificador. Utiliza operações de upsampling e camadas convolucionais para aumentar a resolução da imagem. As camadas do decodificador incluem:

- **UpSampling 1:**
  - 1 camada Conv2DTranspose com 512 filtros e kernel 2x2
  - 2 camadas Conv2D com 512 filtros e kernel 3x3, ativação ReLU

- **UpSampling 2:**
  - 1 camada Conv2DTranspose com 256 filtros e kernel 2x2
  - 2 camadas Conv2D com 256 filtros e kernel 3x3, ativação ReLU

- **UpSampling 3:**
  - 1 camada Conv2DTranspose com 128 filtros e kernel 2x2
  - 2 camadas Conv2D com 128 filtros e kernel 3x3, ativação ReLU

- **UpSampling 4:**
  - 1 camada Conv2DTranspose com 64 filtros e kernel 2x2
  - 2 camadas Conv2D com 64 filtros e kernel 3x3, ativação ReLU


### Saída (Output)
A última camada do decodificador é uma camada Conv2D com um único filtro e função de ativação sigmoid, produzindo a máscara segmentada binária.

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, concatenate, Cropping2D, ZeroPadding2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def unet_with_vgg16_backbone(input_size=(120, 120, 3)):
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=input_size)

    # Freeze the VGG16 layers
    for layer in vgg16.layers:
        layer.trainable = False

    # Encoder
    s1 = vgg16.get_layer("block1_conv2").output
    s2 = vgg16.get_layer("block2_conv2").output
    s3 = vgg16.get_layer("block3_conv3").output
    s4 = vgg16.get_layer("block4_conv3").output
    b1 = vgg16.get_layer("block5_conv3").output

    # Decoder
    u6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(b1)
    if u6.shape[1] != s4.shape[1] or u6.shape[2] != s4.shape[2]:
        u6 = ZeroPadding2D(((0, s4.shape[1] - u6.shape[1]), (0, s4.shape[2] - u6.shape[2])))(u6)
    u6 = concatenate([u6, s4])

    u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(u6)
    if u7.shape[1] != s3.shape[1] or u7.shape[2] != s3.shape[2]:
        u7 = ZeroPadding2D(((0, s3.shape[1] - u7.shape[1]), (0, s3.shape[2] - u7.shape[2])))(u7)
    u7 = concatenate([u7, s3])

    u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(u7)
    if u8.shape[1] != s2.shape[1] or u8.shape[2] != s2.shape[2]:
        u8 = ZeroPadding2D(((0, s2.shape[1] - u8.shape[1]), (0, s2.shape[2] - u8.shape[2])))(u8)
    u8 = concatenate([u8, s2])

    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(u8)
    if u9.shape[1] != s1.shape[1] or u9.shape[2] != s1.shape[2]:
        u9 = ZeroPadding2D(((0, s1.shape[1] - u9.shape[1]), (0, s1.shape[2] - u9.shape[2])))(u9)
    u9 = concatenate([u9, s1])

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(u9)

    model = Model(inputs=vgg16.input, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Criar o modelo
model = unet_with_vgg16_backbone(input_size=(120, 120, 3))


## Divisão dos Dados e Treinamento
Dividimos os dados em conjuntos de treino e teste e treinamos o modelo U-Net com VGG16.

In [ ]:
# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.1, random_state=42)


In [ ]:
# Criar o modelo
model = unet_with_vgg16_backbone(input_size=(120, 120, 3))

In [ ]:
# Treinar o modelo
results = model.fit(X_train, y_train, validation_split=0.1, batch_size=1, epochs=40)


In [ ]:
# Avaliar o modelo
model.evaluate(X_test, y_test, batch_size=1)

## Plotando métricas
Plotamos as métricas de perda e acurácia do modelo durante o treinamento.

In [ ]:
def plot_metrics(history):
    # Plotando a perda (loss)
    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Treino')
    plt.plot(history.history['val_loss'], label='Validação')
    plt.title('Perda')
    plt.ylabel('Perda')
    plt.xlabel('Época')
    plt.legend()

    # Plotando a acurácia (accuracy)
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Treino')
    plt.plot(history.history['val_accuracy'], label='Validação')
    plt.title('Acurácia')
    plt.ylabel('Acurácia')
    plt.xlabel('Época')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_metrics(results)

## Plotando resultados
Plotamos as imagens de entrada, as máscaras reais e as máscaras preditas pelo modelo.

In [ ]:
# Função para plotar os resultados
def plot_results(X, y, model, ix=None):
    """Plotar a imagem, a máscara real e a máscara predita.

    Args:
    - X: array de imagens de entrada.
    - y: array de máscaras reais.
    - model: modelo treinado U-Net.
    - ix: índice da imagem a ser plotada. Se None, seleciona uma imagem aleatoriamente.
    """
    if ix is None:
        ix = np.random.randint(0, len(X))

    fig, ax = plt.subplots(1, 3, figsize=(20, 10))
    ax[0].imshow(X[ix], cmap='gray')
    ax[0].title.set_text('Imagem Original')
    ax[0].axis('off')

    ax[1].imshow(y[ix].squeeze(), cmap='gray')  # Squeeze para remover dimensões extras se houver
    ax[1].title.set_text('Máscara Real')
    ax[1].axis('off')

    # Fazer a predição usando o modelo
    pred = model.predict(X[ix:ix+1])
    pred = (pred > 0.5).astype(np.float32)  # Binarizar a predição

    ax[2].imshow(pred.squeeze(), cmap='gray')  # Squeeze para garantir que está em 2D
    ax[2].title.set_text('Máscara Predita')
    ax[2].axis('off')

    plt.show()

In [ ]:
index = None
plot_results(X_test, y_test, model, ix=index)

## Instruções para Obtenção do Modelo

### Passo 1: Aquisição dos Dados
Os dados utilizados neste exemplo são imagens e máscaras que podem ser carregadas a partir de um diretório local ou de uma unidade do Google Drive. Certifique-se de que as imagens e as máscaras estejam no formato PNG e que as máscaras sejam imagens em escala de cinza (grayscale).


### Passo 2: Configuração do Ambiente
Para utilizar a GPU no Google Colab:

1. Vá para `Tempo de execução` ou `Runtime` no menu.
2. Selecione `Alterar tipo de tempo de execução` ou `Change runtime`.
3. Na janela que abrir, escolha `GPU` no menu suspenso `Acelerador de hardware`, recomendamos o uso da A100 pela capacidade mista de CPU e GPU.

Após realizar esses passos, a GPU será habilitada para o notebook.

### Passo 3: Preparação dos Dados
Os dados devem ser normalizados e divididos em conjuntos de treino e teste. As imagens são normalizadas para o intervalo [0, 1], e as máscaras são binarizadas (valores acima de 0.5 são considerados 1 e abaixo são 0).

### Passo 4: Treinamento do Modelo
O modelo U-Net com backbone VGG16 é treinado utilizando os dados preparados. Ajuste os hiperparâmetros como número de épocas, tamanho do lote e taxa de aprendizado conforme necessário.

### Passo 5: Avaliação do Modelo
Após o treinamento, o modelo é avaliado no conjunto de teste para medir seu desempenho. Métricas como a perda e a acurácia são plotadas para análise.


### Passo 6: Refinamento do Modelo
Se o desempenho do modelo não for satisfatório, existem as seguintes estratégias de refinamento:
1. **Ajuste de Hiperparâmetros:** Modificar a taxa de aprendizado (learning rate ou `lr`), o número de épocas (`epochs`), o tamanho do lote (`batch_size`), etc.
2. **Aumento de Dados (Data Augmentation):** Utilize técnicas de aumento de dados para gerar mais exemplos de treino, como rotações, zoom, translações, etc.
3. **Arquitetura do Modelo:** Modificar a arquitetura do modelo, adicionando mais camadas ou unidades.
4. **Regularização:** Adicionar camadas de Dropout ou ajuste a regularização L2 para evitar overfitting.
5. **Treinamento com Mais Dados:** Adicionar mais dados de treino para melhorar a generalização do modelo.

# Instruções para deploy em um serviço em nuvem

As seguintes instruções fornecem um guia para preparar, fazer upload e fazer o deploy do modelo de segmentação no AWS SageMaker.

### 1. Preparação do ambiente

Instale a AWS CLI e configure suas credenciais:

In [ ]:
pip install awscli
aws configure

Instale o SDK do SageMaker:

In [ ]:
pip install sagemaker

### 2. Preparar o notebook

Certifique-se que todas as dependências do notebook estão instaladas:

In [ ]:
pip install numpy pandas scikit-learn matplotlib tensorflow

### 3. Preparação do modelo

Após o treinamento do modelo, salve-o no formato .h5:

In [ ]:
model.save('segmentation_model.h5')

Após isso, carregue o arquivo do modelo salvo para um bucket no S3:

In [ ]:
aws s3 cp segmentation_model.h5 s3://nome-do-bucket/modelos/segmentation_model.h5

### 4. Deploy no SageMaker

Crie um arquivo inference.py e coloque o seguinte código que contém a lógica de carregamento do modelo e inferência:

In [ ]:
import json
import numpy as np
from tensorflow.keras.models import load_model

def model_fn(model_dir):
    model = load_model(f'{model_dir}/segmentation_model.h5')
    return model

def input_fn(request_body, request_content_type):
    if request_content_type == 'application/json':
        return np.array(json.loads(request_body)['instances'])
    else:
        raise ValueError('Esse modelo apenas suporta JSON')

def predict_fn(input_data, model):
    predictions = model.predict(input_data)
    return predictions

def output_fn(prediction, response_content_type):
    return json.dumps({'predições': prediction.tolist()})

Utilize o SDK do SageMaker para criar um endpoint para o modelo:

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlowModel

sagemaker_session = sagemaker.Session()
role = 'arn:aws:iam::sua-conta:permissao/sagemaker-permissao'

model = TensorFlowModel(model_data='s3://nome-bucket/modelos/segmentation_model.h5',
                        role=role,
                        entry_point='inference.py',
                        framework_version='2.3',
                        sagemaker_session=sagemaker_session)

predictor = model.deploy(initial_instance_count=1,
                         instance_type='ml.m4.xlarge')

### 5. Teste e validação

Envie uma requisição de teste ao endpoint para garantir que tudo esteja funcionando corretamente:

In [ ]:
import json
import numpy as np
import boto3

runtime = boto3.client('runtime.sagemaker')
payload = json.dumps({'instances': np.random.rand(1, 224, 224, 3).tolist()})

response = runtime.invoke_endpoint(EndpointName=predictor.endpoint_name,
                                   ContentType='application/json',
                                   Body=payload)

result = json.loads(response['Body'].read().decode())
print(result)

### 6. Limpeza

Para evitar custos desnecessários, remova os recursos quando terminar:

In [ ]:
predictor.delete_endpoint()